In [3]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
%matplotlib inline

### Challenge baseline

In [4]:
df = pd.read_csv('/data_mlhep2017/episode1/DS_1_train.csv', index_col=0)

df.head()

/opt/conda/lib/python3.6/site-packages/numpy/lib/arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,event_id,X,Y,Z,TX,TY,chi2,signal
index,,,,,,,,
0,-999,42720.238281,75152.406250,32325.0,-0.475539,0.405506,2.652586,0.0
1,-999,46023.816406,37043.359375,21981.0,0.330822,-0.013988,2.497527,0.0
2,-999,52706.117188,33872.730469,16809.0,0.342634,-0.058724,2.120421,0.0
3,-999,47334.101562,9698.531250,60771.0,0.007868,-0.632822,0.618535,0.0
4,-999,67380.835938,19137.265625,37497.0,-0.212537,-0.192243,2.799303,0.0


### Build a train dataset
Ideally, you could design features here that improve model performance. 

Baseline, however, simply takes some of the features while also balansing classes.

In [5]:
print(df[df.signal==True])

         event_id             X             Y        Z        TX        TY  \
index                                                                        
29            411  53290.710938  22941.925781  63357.0 -0.258045  0.710882   
32          27354  36921.367188  21285.517578  72408.0 -0.924607  0.819380   
37           5586  43593.214844  56103.648438  45255.0 -0.330734 -0.467480   
39           3009  54343.628906  49445.316406  42669.0  0.242685 -0.080539   
41            739  41051.804688  34390.445312  60771.0  0.035552 -0.042680   
54          11904  64761.464844  17699.648438  60771.0  0.240346 -0.219322   
59          12128  51462.656250  68327.703125  62064.0 -0.219973  0.124991   
72          32733  59579.835938  51586.804688  46548.0 -0.673363 -0.461440   
78           9240  48817.488281  23558.681641  54306.0  0.405901 -0.005157   
89           5207  58450.824219  22339.125000  65943.0  0.149967 -0.249837   
97           7027  60456.890625   5661.429688  62064.0  0.499470

In [6]:
#takes the chi2 of all signal tracks + the value of the signal variable
print(df[df.signal==True].values[:,6:8])

[[ 1.84655619  1.        ]
 [ 0.84937239  1.        ]
 [ 0.64618766  1.        ]
 ..., 
 [ 0.89810956  1.        ]
 [ 1.14396203  1.        ]
 [ 0.77642524  1.        ]]


In [7]:
#takes the chi2 of the first 50000 bkg tracks + the value of the signal variable
print(df[df.signal==False].values[:50000,6:8])

[[ 2.65258646  0.        ]
 [ 2.49752712  0.        ]
 [ 2.12042117  0.        ]
 ..., 
 [ 1.75582957  0.        ]
 [ 2.49099779  0.        ]
 [ 1.22334659  0.        ]]


In [8]:
#signal is more scarse => 
train_df = np.concatenate([df[df.signal == True].values[:, 6:8],
                           df[df.signal == False].values[:50000, 6:8]])
#values[,slices some of the features]

print("length array signal+bkg: ", len(train_df))
print("train_df: ", train_df[0:10])

length array signal+bkg:  101018
train_df:  [[ 1.84655619  1.        ]
 [ 0.84937239  1.        ]
 [ 0.64618766  1.        ]
 [ 0.41360524  1.        ]
 [ 0.67065531  1.        ]
 [ 1.42008829  1.        ]
 [ 0.36800182  1.        ]
 [ 1.25551558  1.        ]
 [ 0.75633907  1.        ]
 [ 1.1757319   1.        ]]


In [9]:
#
X_train = train_df[:, 0].reshape(-1, 1)
print (X_train[0:10])
y_train =  np.array(train_df[:, 1], dtype = int)
print (y_train[0:10])

[[ 1.84655619]
 [ 0.84937239]
 [ 0.64618766]
 [ 0.41360524]
 [ 0.67065531]
 [ 1.42008829]
 [ 0.36800182]
 [ 1.25551558]
 [ 0.75633907]
 [ 1.1757319 ]]
[1 1 1 1 1 1 1 1 1 1]


### Train a model

In [10]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_depth = 1)

clf.fit(X_train,y_train)

#to define best classifier, look at accuracy or ROC curve


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

### Prediction

In [11]:
df_test = pd.read_csv('/data_mlhep2017/episode1/DS_1_test.csv', index_col=0)
df_test.head()

/opt/conda/lib/python3.6/site-packages/numpy/lib/arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,X,Y,Z,TX,TY,chi2
index,,,,,,
0,37259.390625,62516.367188,63357.0,-0.475465,-0.250186,1.405402
1,41426.312500,57520.265625,25860.0,-0.213876,0.155208,2.541856
2,40752.921875,54653.265625,46548.0,0.044215,-0.333426,2.389629
3,29293.980469,23582.425781,14223.0,-0.265671,-0.137444,2.402328
4,49538.128906,63314.617188,31032.0,-0.063467,-0.550298,1.223460


In [12]:
prediction = clf.predict_proba(df_test.values[:, 5].reshape(-1, 1))[:, 1]

### Save results

In [14]:
baseline = pd.DataFrame(prediction, columns=['Prediction'])
baseline.index.name = 'Id'
baseline.to_csv('Mybaseline.csv', header=True)

You can now download the baseline.csv in jupyter file browser and send it to Kaggle.